In [17]:
%pylab
%matplotlib inline
from PIL import Image
import numpy as np

Using matplotlib backend: QtAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Funktion, die die YCbCr Werte ausrechnet. Formel für JPEG-Konversion, daher muss das Bild idealerweise als .jpg gespeichert werden.
Wikipedia: JFIF usage of JPEG supports a modified Rec. 601 Y′CbCr where Y′, CB and CR have the full 8-bit range of [0...255].[9] Below are the conversion equations expressed to six decimal digits of precision. (For ideal equations, see ITU-T T.871.[10]) Note that for the following formulae, the range of each input (R,G,B) is also the full 8-bit range of [0...255].


In [18]:
def rgb_to_ycbcr(r, g, b): # in (0,255) range
    y = .299*r + .587*g + .114*b
    cb = 128 -.168736*r -.331364*g + .5*b
    cr = 128 +.5*r - .418688*g - .081312*b
    return y, cb, cr

rgb_to_ycbcr(37, 16, 111)

(33.108999999999995, 171.954944, 130.77536)

Fancy Funktion für die RGB-to-YCbCr Konversion
Erklärung der Fancy Funktion:
- isistance: Bild muss als PIL Image eingelesen werden (open Funktion)
- RGB channels werden durch split Methode in 3 Banden (r, g und b) gesplittet
- leere Listen werden erstellt
- rd (von red), gn (green), bl (blue) 
- zip Methode erstellt Tupeln von den R, G und B Banden --> getdata extrahiert die Werte der jeweiligen Banden (R, G, B) --> so werden die RGB Werte für jedes Pixel in ein Tupel reingepackt
- Diese RGB Werte werden zu YCbCr konvertiert --> 
- Die Werte der R, G und B Banden werden durch die YCbCr-Werte ersetzt. Achtung! Es bleiben trotzdem als r, g, b benannt, weil die 3 Banden am Anfang so definiert wurden
- Deshalb wird bei merge Funktion r, g, b eingegeben, aber Mode muss YCbCr sein

In [28]:
def YCbCrColor(img):
    if isinstance(img, Image.Image):
        r,g,b = img.split()
        Ydat = []
        Cbdat = []
        Crdat = []
        for rd, gn, bl in zip(r.getdata(), g.getdata(), b.getdata()):
            Y,Cb,Cr = rgb_to_ycbcr(rd, gn, bl)
            Ydat.append(int(Y))
            Cbdat.append(int(Cb))
            Crdat.append(int(Cr))
        r.putdata(Ydat)
        g.putdata(Cbdat)
        b.putdata(Crdat)
        return Image.merge("YCbCr", (r,g,b))
    else:
        return None

YCbCr Bilder sind für .JPG gedacht, deshalb in diesem Format speichern

In [35]:
RGB_image = Image.open("../data/Cell_Nuclei.jpg")
YCbCr_image = YCbCrColor(RGB_image)
YCbCr_image.show() #anzeigen

In [12]:
print(YCbCr_image.getpixel((0,0)), RGB_image.getpixel((0,0)))

(1, 126, 128) (2, 2, 0)


In [30]:
YCbCr_image_array_fancyfunction = np.asarray(YCbCr_image) #in array umwandeln

Nicht so fancy Funktion (aber vielleicht verständlicher) mit array, das Output-Bild ist normal was die Farben angeht, aber an manchen Stellen verpixelter als die Fancy Funktion
Erklärung Funktion mit array:
- Bild wird in array umgewandelt
- Leere Liste wird erstellt
- für jedes Pixel werden die RGB Werte aus dem RGB_array in YCbCr Werte umgewandelt
- YCbCr Werte werden für jedes Pixel der Liste eingefügt
- Liste mit YCbCr Werten wird zu 3D-Array mit den gleichem Shape wie das Bild umgewandelt

In [33]:
def image_ycbcr_conversion_array(RGB_image):
    RGB_array = np.array(RGB_image)
    width, height = RGB_image.size
    YCbCr_list = []
    for x in range(width):
        for y in range(height):
            r, g, b = RGB_array[x,y]
            Y, Cb, Cr = rgb_to_ycbcr(r, g, b)
            listYCbCr = [int(Y), int(Cb), int(Cr)]
            YCbCr_list.append(listYCbCr)
    YCbCr_array = np.asarray(YCbCr_list, dtype=uint8).reshape(width, height, 3)
    return YCbCr_array

In [39]:
YCbCr_image_array_arrayfunction = image_ycbcr_conversion_array(RGB_image)
Image.fromarray(YCbCr_image_array_arrayfunction, "YCbCr").save("../data/YCbCr_CellNuclei_array.jpg") #anzeigen